# `nn.utils.prune` 모듈로 BERT 파라미터 Pruning 해보기

최근 **자연어 처리**를 비롯한 많은 분야에서 태스크 성능을 높이기 위해 모델의 크기를 늘리는 연구를 하고 있습니다. 그리고 이를 통해 비약적인 성능의 개선을 이끌어 내긴 했지만, 지나치게 커진 모델 크기로 인해 **실제 어플리케이션**에 배포를 하기 어려워졌다는 **사이드 이펙트**가 생겨났습니다.

그러나 이전부터 모델의 크기 문제를 타파하기 위한 해결책으로 다양한 모델 압축 기법들이 연구되어 왔고, 모델이 예측 값을 내는데 큰 영향을 미치지 않는 파라미터들을 가지치기 하는 **Pruning**도 이러한 모델 압축 기법 중 하나입니다.

본 튜토리얼에서는 [**Pruning Tutorial**](https://pytorch.org/tutorials/intermediate/pruning_tutorial.html#pruning-a-module)을 참조해 `torch.nn.utils.prune` 모듈을 활용한 **BERT** 파라미터 **Pruning** 사용 예에 대해 살펴보도록 합니다. 

**BERT**의 경우 **Transformer** 모델을 기반으로 하지만, 최근 많은 연구들에서 **Multi-Head Attention** 혹은 더 나아가 무의미한 **Attention map**의 효능에 대해 의문을 품고 있습니다. 그리고 이에 따라 헤드의 갯수를 줄인다거나, 어텐션 매트릭스를 **Pruning** 한다거나 등의 다양한 시도가 행해지고 있습니다. 

우리는 이 중, 어텐션 매트릭스를 **Pruning** 해보는 시간을 본 튜토리얼을 통해 가져볼 것입니다.

_p.s. Model Compression 기법들은 BERT와 같은 큰 모델의 등장 이전에도 꾸준히 연구되어 오던 분야입니다._

먼저 튜토리얼에 사용될 라이브러리들을 임포트합니다. 본 튜토리얼을 수행하기 위해서는 **1.4.0** 버전 이상의 `torch`가 필요합니다.

In [ ]:
import torch
import torch.nn.utils.prune as prune

torch.__version__

## 모델 로드

본 튜토리얼에서는 **SKT Brain**이 훈련시키고 [@monologg](https://github.com/monologg)님이 Hugging face 사의 [**transformer**](https://github.com/huggingface/transformers) 라이브러리에 등록한 **KoBERT** 모델을 활용합니다. 

해당 모델에 대해 더 자세한 설명을 원하시는 분은 [**본 저장소**](https://github.com/SKTBrain/KoBERT)를 참조하시면 좋을 것 같습니다.

이제 모델을 불러오도록 합니다. 사전에 모델이 설치되어 있지 않았다면 아래 명령어를 통해 모델이 다운로드 및 로드가 됩니다.

In [2]:
from transformers import BertModel
model = BertModel.from_pretrained('monologg/kobert')

## 모델 조사

**Pruning**이 아직 적용되지 않은 **KoBERT** 모델을 먼저 살펴보도록 합니다.

해당 모델은 아래에서 살펴볼 수 있듯 `embeddings`, `pooler`를 비롯한 12개의 `BertEncoder` 모듈들로 구성되어 있습니다.

In [3]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

우리는 위 모듈들 중**인코더 레이어**들에만 관심이 있으므로 해당 모듈만 분석해봅시다.

In [4]:
model.encoder.layer

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (1): BertLayer(
    (attention): BertAttention(
      (self)

앞으로 나올 예제 진행을 위해 모든 레이어를 사용할 필요는 없습니다.

먼저, 12개의 인코더 레이어 중 **최하단 레이어**만 살펴보도록 합시다.

In [5]:
model.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

마지막으로 우리는 위 구성 중 `attention` 모듈에만 관심이 있으므로 `attention` 모듈을 살펴봅시다.

In [6]:
model.encoder.layer[0].attention.self

BertSelfAttention(
  (query): Linear(in_features=768, out_features=768, bias=True)
  (key): Linear(in_features=768, out_features=768, bias=True)
  (value): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

`BertSelfAttention`은 `query`, `key`, `value` 그리고 `dropout`으로 구성되어 있습니다. 

이제 q, k, v에 **Pruning**을 진행하기 앞서 `key`에 **Pruning**을 적용하는 연습을 해보기 위해 실험 모듈을 `key`로 지정합니다.

현재 `key`의 `named_parameters()` 내에는 `weight`와 `bias`가 존재하며, `named_buffers()`에는 아무것도 존재하지 않습니다.

<br/>

_cf. What is the difference between register_parameter and register_buffer in PyTorch?_

> This is typically used to register a buffer that should not to be considered a model parameter. <br/> For example, BatchNorm’s running_mean is not a parameter, but is part of the persistent state

In [7]:
module = model.encoder.layer[0].attention.self.key

In [8]:
list(module.named_parameters())

[('weight', Parameter containing:
  tensor([[ 0.0267,  0.0427, -0.0713,  ..., -0.0344,  0.0055,  0.0277],
          [ 0.0228,  0.0752, -0.0429,  ...,  0.0860,  0.1789,  0.0289],
          [-0.0079, -0.0309, -0.0990,  ..., -0.0316,  0.0335, -0.0635],
          ...,
          [ 0.0040, -0.0065, -0.0924,  ..., -0.0338, -0.0250, -0.1278],
          [ 0.0134,  0.0512,  0.0694,  ..., -0.0096, -0.0297,  0.0294],
          [-0.0243, -0.0592, -0.0535,  ..., -0.0318, -0.0714,  0.0376]],
         requires_grad=True)), ('bias', Parameter containing:
  tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
          -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
           8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -1.0340e-08,
          -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
          -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
          -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-08,
      

In [9]:
list(module.named_buffers())

[]

## 모델 Pruning

모듈을 Pruning 하기 위해서는 **자신이 직접 구현한 기법** 혹은 `torch.nn.utils.prune` 내에 존재하는 Pruning 기법 중 하나를 활용해야 합니다. 

그리고 해당 기법을 활용해 Pruning을 적용할 **모듈명**과 **파라미터명**을 상세해줍니다.

아래 예제에서는 `key` 내 `weight` 중 임의로 **30%**의 파라미터에 Pruning을 적용합니다. 

이때 첫 번째 인자로는 **모듈**이, 두 번째인  `name`의 인자로는 Pruning을 적용할 **모듈 내 파라미터명**이, 그리고 마지막 `amount`의 인자로는 **0 에서 1 사이의 소수** (Pruning이 적용될 퍼센티지) 혹은 **양의 정수**(Pruning을 적용할 파라미터 개수)가 사용됩니다.

In [10]:
prune.random_unstructured(module, name="weight", amount=0.3)

Linear(in_features=768, out_features=768, bias=True)

`utils.prune` 모듈의 **Pruning**은 `weight`를 모듈 내 파라미터에서 제거한 후, `weight_orig`로 대체함으로써 적용됩니다. 

이때, `weight_orig`는 Pruning이 적용되지 않은 텐서입니다.

In [11]:
list(module.named_parameters())

[('bias', Parameter containing:
  tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
          -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
           8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -1.0340e-08,
          -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
          -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
          -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-08,
           1.1002e-07, -6.4796e-08, -1.4740e-07, -1.9011e-08,  2.2245e-07,
          -1.4329e-07, -2.8459e-08,  7.1192e-08, -5.4687e-08, -1.0020e-07,
          -1.4137e-07, -4.4915e-08, -2.6025e-08, -2.9728e-07, -8.7707e-08,
          -3.9435e-08, -5.6708e-08, -4.0600e-08,  1.3973e-07, -2.7347e-07,
          -1.3187e-07,  1.3556e-08, -1.4675e-08,  4.6401e-08, -5.2453e-08,
          -6.1351e-08, -8.3668e-08,  1.2962e-07,  2.1639e-07,  7.9439e-08,
          -4.9919e-08, -6.5474e-08,  1.9281e-08, -4.2483e-08,  1.639

이제 모듈의 `named_buffers()`를 출력하면 Pruning 기법에 의해 생성된 **Pruning mask**가 `weight_mask`라는 이름으로 생성된 것을 확인할 수 있습니다.

In [12]:
list(module.named_buffers())

[('weight_mask', tensor([[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 0., 1.,  ..., 1., 1., 1.],
          [0., 1., 1.,  ..., 0., 1., 0.],
          ...,
          [1., 0., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 0.],
          [1., 1., 1.,  ..., 1., 1., 1.]]))]

PyTorch에서 모델에 순전파를 적용하기 위해서는 `weight` 속성이 존재해야 합니다.

`weight`는 원래의 `weight_orig` 값에 앞서 생성한 **Pruning mask**를 적용해 계산합니다.

그리고 해당 계산 결과가 `weight`라는 이름의 **속성**으로 모듈에 저장되게 됩니다.

이제 `weight`는 더 이상 모듈의 **파라미터**로 관리되는 것이 아니라 모듈의 **속성(attribute)** 값으로 관리되게 되는 것입니다.

In [13]:
module.weight

tensor([[ 0.0267,  0.0427, -0.0713,  ..., -0.0344,  0.0055,  0.0277],
        [ 0.0228,  0.0000, -0.0429,  ...,  0.0860,  0.1789,  0.0289],
        [-0.0000, -0.0309, -0.0990,  ..., -0.0000,  0.0335, -0.0000],
        ...,
        [ 0.0040, -0.0000, -0.0924,  ..., -0.0338, -0.0250, -0.1278],
        [ 0.0134,  0.0512,  0.0694,  ..., -0.0096, -0.0297,  0.0000],
        [-0.0243, -0.0592, -0.0535,  ..., -0.0318, -0.0714,  0.0376]],
       grad_fn=<MulBackward0>)

In [14]:
module.weight.size(), (module.weight == 0).sum()  # 30% 파라미터가 Pruned !

(torch.Size([768, 768]), tensor(176947))

마지막으로 `nn.utils.prune`은 앞서 적용한 **Pruning**을 순전파 이전에 적용하기 위해 `forward_pre_hooks`라는 속성을 사용합니다. 

우리는 앞서 `weight` 파라미터에만 **Pruning**을 적용했기 때문에 아래와 같이 하나의 **pre_hook**이 생성된 것을 확인할 수 있습니다.

In [15]:
module._forward_pre_hooks

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured at 0x1ed7fb16a90>)])

모듈의 파라미터, 버퍼, 훅 그리고 속성이 어떻게 변하는지 다시 한 번 확인하기 위해, `key`의 `bias`에도 **Pruning**을 적용해보도록 합시다. 

이번에는 `bias`의 50개의 파라미터에 **Pruning**을 적용해보도록 합니다. 

참고로 `l1_unstructured`는 인자로 받은 파라미터에서 **L1 노름** 기준으로 가장 영향력이 작은 `amount`개의 파라미터를 **Pruning**하도록 구현되어 있습니다.

In [16]:
prune.l1_unstructured(module, name='bias', amount=50)

Linear(in_features=768, out_features=768, bias=True)

이제 우리는 모듈의 `named_parameters()`에 `weight_orig` 뿐만 아니라 `bias_orig`가 함께 포함되어 있음을 예상할 수 있습니다. 

그리고 `named_buffers()`에는 `weight_mask`와 더불어 `bias_mask`가 포함되어 있겠죠. 

**Pruning**이 적용된 두 텐서는 앞서 언급한 것과 마찬가지로 이제 모듈의 **속성**으로서 관리되게 됩니다.

In [17]:
list(module.named_parameters())

[('weight_orig', Parameter containing:
  tensor([[ 0.0267,  0.0427, -0.0713,  ..., -0.0344,  0.0055,  0.0277],
          [ 0.0228,  0.0752, -0.0429,  ...,  0.0860,  0.1789,  0.0289],
          [-0.0079, -0.0309, -0.0990,  ..., -0.0316,  0.0335, -0.0635],
          ...,
          [ 0.0040, -0.0065, -0.0924,  ..., -0.0338, -0.0250, -0.1278],
          [ 0.0134,  0.0512,  0.0694,  ..., -0.0096, -0.0297,  0.0294],
          [-0.0243, -0.0592, -0.0535,  ..., -0.0318, -0.0714,  0.0376]],
         requires_grad=True)), ('bias_orig', Parameter containing:
  tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
          -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
           8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -1.0340e-08,
          -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
          -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
          -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-

In [18]:
list(module.named_buffers())

[('weight_mask', tensor([[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 0., 1.,  ..., 1., 1., 1.],
          [0., 1., 1.,  ..., 0., 1., 0.],
          ...,
          [1., 0., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 0.],
          [1., 1., 1.,  ..., 1., 1., 1.]])),
 ('bias_mask',
  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 0., 1., 1., 1.

In [19]:
module.bias

tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
        -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
         8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -0.0000e+00,
        -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
        -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
        -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-08,
         1.1002e-07, -6.4796e-08, -1.4740e-07, -1.9011e-08,  2.2245e-07,
        -1.4329e-07, -2.8459e-08,  7.1192e-08, -5.4687e-08, -1.0020e-07,
        -1.4137e-07, -4.4915e-08, -2.6025e-08, -2.9728e-07, -8.7707e-08,
        -3.9435e-08, -5.6708e-08, -4.0600e-08,  1.3973e-07, -2.7347e-07,
        -1.3187e-07,  1.3556e-08, -1.4675e-08,  4.6401e-08, -5.2453e-08,
        -6.1351e-08, -8.3668e-08,  1.2962e-07,  2.1639e-07,  7.9439e-08,
        -4.9919e-08, -6.5474e-08,  1.9281e-08, -4.2483e-08,  1.6391e-07,
        -3.8939e-07, -1.1450e-07, -3.8202e-07, -2.1

In [20]:
module.bias.size(), (module.bias == 0).sum()

(torch.Size([768]), tensor(50))

모듈의 `_forward_pre_hooks`에도 이제 2개의 **pre_hook**이 존재하는 것을 확인할 수 있습니다.

In [21]:
module._forward_pre_hooks

OrderedDict([(0, <torch.nn.utils.prune.RandomUnstructured at 0x1ed7fb16a90>),
             (1, <torch.nn.utils.prune.L1Unstructured at 0x1ed7fb2ccc0>)])

## Pruning 중첩

모듈 내 같은 파라미터에 Pruning을 여러 번 적용할 수도 있습니다. Pruning을 여러 번 적용한다는 것은 다양한 **Pruning mask**를 중첩해서 사용하겠다는 의미입니다. 그리고 중첩된 마스크 결과 값은 `PruningContainer` 객체의 `compute_mask` 메서드에 의해 관리됩니다.

아래 예제는 이전에 Pruning을 적용한 `key`의 `weight`에 또 다른 Pruning을 적용하는 예를 보여줍니다. `ln_structured`는 텐서의 `dim`**-th axis**에서 L`n` 노름을 기준으로 영향력이 작은 `amount`만큼의 파라미터를 **Pruning** 하는 기법입니다.

In [22]:
prune.ln_structured(module, name='weight', amount=0.3, n=2, dim=1)

Linear(in_features=768, out_features=768, bias=True)

이제 모듈의 `weight`는 앞서 적용한 `random_unstructured`와 `ln_structured`가 함께 적용된 `weight_mask`를 통해 중첩 **Pruning**이 적용되었습니다. 

앞서 출력한 `weight_mask` 보다 **Pruning**이 적용될 0의 갯수가 많아진 것을 확인할 수 있습니다.

In [23]:
list(module.named_buffers())[0][1]

tensor([[1., 0., 1.,  ..., 0., 0., 1.],
        [1., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 1.,  ..., 0., 0., 1.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 1.]])

In [24]:
module.weight

tensor([[ 0.0267,  0.0000, -0.0713,  ..., -0.0000,  0.0000,  0.0277],
        [ 0.0228,  0.0000, -0.0429,  ...,  0.0000,  0.0000,  0.0289],
        [-0.0000, -0.0000, -0.0990,  ..., -0.0000,  0.0000, -0.0000],
        ...,
        [ 0.0040, -0.0000, -0.0924,  ..., -0.0000, -0.0000, -0.1278],
        [ 0.0134,  0.0000,  0.0694,  ..., -0.0000, -0.0000,  0.0000],
        [-0.0243, -0.0000, -0.0535,  ..., -0.0000, -0.0000,  0.0376]],
       grad_fn=<MulBackward0>)

이제 `weight`에 중첩 **Pruning**을 적용하는 객체는 `torch.nn.utils.prune.PruningContainer`가 됩니다. 

해당 컨테이너는 `weight` 파라미터에 적용된 **Pruning** 기법들의 내역을 저장합니다.

In [25]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == 'weight':
        break

hook

In [26]:
list(hook)

## Pruning 모델 시리얼라이즈

마스크 버퍼(`_mask`)와 파라미터의 원래 값(`_orig`) 등 **Pruning**에 사용되는 모든 관련된 텐서들은 각 모듈의 `state_dict`에 저장되기 때문에 쉽게 저장 및 시리얼라이즈 될 수 있습니다.

In [27]:
module.state_dict().keys()

odict_keys(['weight_orig', 'bias_orig', 'weight_mask', 'bias_mask'])

## Pruning 영구 적용

우리는 `weight_orig`와 `weight_mask`, 그리고 **pre_hook** 등 **Pruning**에 사용된 모듈들을 제거해 Pruning을 영구적으로 적용할 수 있습니다. 

그리고 이를 위해서는 `remove` 메서드를 사용해야 합니다.

In [28]:
list(module.named_parameters())

[('weight_orig', Parameter containing:
  tensor([[ 0.0267,  0.0427, -0.0713,  ..., -0.0344,  0.0055,  0.0277],
          [ 0.0228,  0.0752, -0.0429,  ...,  0.0860,  0.1789,  0.0289],
          [-0.0079, -0.0309, -0.0990,  ..., -0.0316,  0.0335, -0.0635],
          ...,
          [ 0.0040, -0.0065, -0.0924,  ..., -0.0338, -0.0250, -0.1278],
          [ 0.0134,  0.0512,  0.0694,  ..., -0.0096, -0.0297,  0.0294],
          [-0.0243, -0.0592, -0.0535,  ..., -0.0318, -0.0714,  0.0376]],
         requires_grad=True)), ('bias_orig', Parameter containing:
  tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
          -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
           8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -1.0340e-08,
          -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
          -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
          -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-

In [29]:
list(module.named_buffers())

[('weight_mask', tensor([[1., 0., 1.,  ..., 0., 0., 1.],
          [1., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 1.,  ..., 0., 0., 1.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 1.]])),
 ('bias_mask',
  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 0., 1., 1., 1.

In [30]:
module.weight

tensor([[ 0.0267,  0.0000, -0.0713,  ..., -0.0000,  0.0000,  0.0277],
        [ 0.0228,  0.0000, -0.0429,  ...,  0.0000,  0.0000,  0.0289],
        [-0.0000, -0.0000, -0.0990,  ..., -0.0000,  0.0000, -0.0000],
        ...,
        [ 0.0040, -0.0000, -0.0924,  ..., -0.0000, -0.0000, -0.1278],
        [ 0.0134,  0.0000,  0.0694,  ..., -0.0000, -0.0000,  0.0000],
        [-0.0243, -0.0000, -0.0535,  ..., -0.0000, -0.0000,  0.0376]],
       grad_fn=<MulBackward0>)

위 코드들에서는 여전히 `weight_orig`와 `weight_mask`가 존재합니다. 이제 이들을 제거하고 **Pruning**을 영구 적용해보도록 합시다.

In [31]:
prune.remove(module, 'weight')
list(module.named_parameters())

[('bias_orig', Parameter containing:
  tensor([ 5.4592e-08,  2.9247e-08, -2.0385e-08, -2.8358e-07, -9.8012e-08,
          -5.8178e-08, -1.3005e-07, -5.6738e-08, -2.4501e-08, -4.8126e-08,
           8.5868e-08, -6.5552e-08, -1.5294e-07, -6.0779e-08, -1.0340e-08,
          -1.0260e-07, -1.4049e-07, -5.1941e-08, -1.7012e-07, -6.5405e-08,
          -1.5491e-07, -1.1515e-07, -2.5359e-08,  4.9711e-08,  1.3979e-07,
          -8.6575e-08, -8.9431e-08,  1.3025e-07, -2.3191e-07, -6.7193e-08,
           1.1002e-07, -6.4796e-08, -1.4740e-07, -1.9011e-08,  2.2245e-07,
          -1.4329e-07, -2.8459e-08,  7.1192e-08, -5.4687e-08, -1.0020e-07,
          -1.4137e-07, -4.4915e-08, -2.6025e-08, -2.9728e-07, -8.7707e-08,
          -3.9435e-08, -5.6708e-08, -4.0600e-08,  1.3973e-07, -2.7347e-07,
          -1.3187e-07,  1.3556e-08, -1.4675e-08,  4.6401e-08, -5.2453e-08,
          -6.1351e-08, -8.3668e-08,  1.2962e-07,  2.1639e-07,  7.9439e-08,
          -4.9919e-08, -6.5474e-08,  1.9281e-08, -4.2483e-08,  

In [32]:
list(module.named_buffers())

[('bias_mask',
  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
 

In [33]:
module._forward_pre_hooks

OrderedDict([(1, <torch.nn.utils.prune.L1Unstructured at 0x1ed7fb2ccc0>)])

이제 `weight`에 Pruning을 적용함으로 인해 생겼던 부산물들: `weight_orig`, `weight_mask` 그리고 **pre_hook**이 모두 제거되고, 

Pruning이 영구 적용된 텐서가 모듈의 파라미터 `weight`를 대체하게 되었습니다.

## 여러 개의 파라미터 Pruning

Pruning 기법과 파라미터를 명세하여 여러 개의 텐서에 동시다발적으로 Pruning을 적용할 수도 있습니다. 

해당 유즈케이스는 아래 코드와 같습니다.

In [34]:
new_model = BertModel.from_pretrained('monologg/kobert')

In [35]:
for name, module in new_model.named_modules():
    if isinstance(module, torch.nn.Embedding):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

In [36]:
dict(new_model.named_buffers()).keys()

dict_keys(['embeddings.word_embeddings.weight_mask', 'embeddings.position_embeddings.weight_mask', 'embeddings.token_type_embeddings.weight_mask', 'encoder.layer.0.attention.self.query.weight_mask', 'encoder.layer.0.attention.self.key.weight_mask', 'encoder.layer.0.attention.self.value.weight_mask', 'encoder.layer.0.attention.output.dense.weight_mask', 'encoder.layer.0.intermediate.dense.weight_mask', 'encoder.layer.0.output.dense.weight_mask', 'encoder.layer.1.attention.self.query.weight_mask', 'encoder.layer.1.attention.self.key.weight_mask', 'encoder.layer.1.attention.self.value.weight_mask', 'encoder.layer.1.attention.output.dense.weight_mask', 'encoder.layer.1.intermediate.dense.weight_mask', 'encoder.layer.1.output.dense.weight_mask', 'encoder.layer.2.attention.self.query.weight_mask', 'encoder.layer.2.attention.self.key.weight_mask', 'encoder.layer.2.attention.self.value.weight_mask', 'encoder.layer.2.attention.output.dense.weight_mask', 'encoder.layer.2.intermediate.dense.weigh

## 글로벌 Pruning

지금까지 저희가 살펴본 예제들은 모델 내 존재하는 텐서들을 개별적으로 가지치기하는 **로컬 Pruning**의 예였습니다. 

그러나 가장 효과적이고 대중적인 Pruning 방법은 모델 전체에 Pruning을 한 번에 적용하는 **글로벌 Pruning** 입니다. 

아래 예제와 같이 모델 전체에 **글로벌 Pruning**을 적용하게 되면 앞선 예제들에서처럼 개별 텐서에서 영향력이 작은 파라미터가 가지치기 하는 것이 아닌, <br/>**모듈 간 연결**에 있어 영향력이 작은 파라미터들이 가지치기 하게 됩니다.

In [37]:
final_model = BertModel.from_pretrained('monologg/kobert')

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (final_model.encoder.layer[i].attention.self.key, 'weight'),
        (final_model.encoder.layer[i].attention.self.query, 'weight'),
        (final_model.encoder.layer[i].attention.self.value, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

In [39]:
for i in range(12):
    print(
        "Sparsity in Layer {}-th key weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.encoder.layer[i].attention.self.key.weight == 0))
            / float(final_model.encoder.layer[i].attention.self.key.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th query weightt: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.encoder.layer[i].attention.self.query.weight == 0))
            / float(final_model.encoder.layer[i].attention.self.query.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th value weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.encoder.layer[i].attention.self.value.weight == 0))
            / float(final_model.encoder.layer[i].attention.self.value.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(final_model.encoder.layer[i].attention.self.key.weight == 0)
    numerator += torch.sum(final_model.encoder.layer[i].attention.self.query.weight == 0)
    numerator += torch.sum(final_model.encoder.layer[i].attention.self.value.weight == 0)

    denominator += final_model.encoder.layer[i].attention.self.key.weight.nelement()
    denominator += final_model.encoder.layer[i].attention.self.query.weight.nelement()
    denominator += final_model.encoder.layer[i].attention.self.value.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th key weight: 18.36%
Sparsity in Layer 1-th query weightt: 18.93%
Sparsity in Layer 1-th value weight: 28.16%

Sparsity in Layer 2-th key weight: 16.87%
Sparsity in Layer 2-th query weightt: 17.30%
Sparsity in Layer 2-th value weight: 30.48%

Sparsity in Layer 3-th key weight: 16.98%
Sparsity in Layer 3-th query weightt: 16.89%
Sparsity in Layer 3-th value weight: 27.35%

Sparsity in Layer 4-th key weight: 17.22%
Sparsity in Layer 4-th query weightt: 17.20%
Sparsity in Layer 4-th value weight: 27.11%

Sparsity in Layer 5-th key weight: 17.35%
Sparsity in Layer 5-th query weightt: 17.37%
Sparsity in Layer 5-th value weight: 26.13%

Sparsity in Layer 6-th key weight: 17.03%
Sparsity in Layer 6-th query weightt: 17.20%
Sparsity in Layer 6-th value weight: 26.90%

Sparsity in Layer 7-th key weight: 16.89%
Sparsity in Layer 7-th query weightt: 16.98%
Sparsity in Layer 7-th value weight: 24.79%

Sparsity in Layer 8-th key weight: 16.84%
Sparsity in Layer 8-th query weigh

위 결과를 통해 모듈 내 각 파라미터의 **Pruning** 비율은 **20%**가 되지 않지만 전체 Sparsity가 20%가 되는 것을 확인할 수 있습니다.

본 튜토리얼에서 살펴본 바와 같이 `utils.prune` 모듈을 활용하면 모델 내 여러 모듈들에 다양한 Pruning 기법을 적용해볼 수 있습니다.

그리고 `utils.prune` 모듈에서 제공하는 추상 클래스를 활용해 본인의 커스텀 Pruning 메서드 또한 손쉽게 작성할 수 있습니다.

해당 모듈은 연구자분들도 많이 사용하는 모듈이라고 하니, 실험 시 활용해보심을 고려해봐도 좋을 것 같아 이번 기회를 통해 소개드렸습니다. 많은 분들께 도움이 되는 튜토리얼이 되었기를 바랍니다!